# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-08bdb103fb-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""
# he author hass a "notable work" property IRI P800. Let's explore its objects
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q35610 wdt:P800 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# canon of Sherlock Holmes is one of the objects. It's IRI is Q2316684. Let's look at its propeties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

#Let's check the instance of property objects
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# We can see it's a book series IRI Q277759. We can write our final query
queryString = """
SELECT DISTINCT ?canon ?name
WHERE {
   # bind something
   ?canon wdt:P31 wd:Q277759 .
   # get the label
   ?canon sc:name ?name.
   FILTER regex(?name, "canon of Sherlock Holmes", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('canon', 'http://www.wikidata.org/entity/Q2316684'), ('name', 'canon of Sherlock Holmes')]
1


Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?canon ?name
WHERE {
   # bind something
   ?canon wdt:P31 wd:Q277759 .
   # get the label
   ?canon sc:name ?name.
   FILTER regex(?name, "canon of Sherlock Holmes", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('canon', 'http://www.wikidata.org/entity/Q2316684'), ('name', 'canon of Sherlock Holmes')]
1


## Task 2
Identify the BGP for film

In [4]:
# write your queries
# let's try to find subjects with "film" in the name
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?s sc:name ?name.
   FILTER regex(?name, "film", "i")
}
LIMIT 20
"""

# The subject "film poster" IRI P3383 might help us because each film should have a poster.
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P3383 ?obj .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""

# Cool! we found some films. Let's take "The Paradine Case" an see if it's an instance of film
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q506638 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# We found the film entity IRI Q11424. Let's see what it's an instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11424 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# No results were returned. let's try the subclass property
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11424 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

# "Moving image" seems like a good super class to use.
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # bind something
   ?film wdt:P279 wd:Q10301427 .
   # get the label
   ?film sc:name ?name.
   FILTER regex(?name, "film", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [5]:
# write your final query
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # bind something
   ?film wdt:P279 wd:Q10301427 .
   # get the label
   ?film sc:name ?name.
   FILTER regex(?name, "film", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 3
Identify the BGP for television series

In [6]:
# write your queries
# let's try to find subjects with "television series" in the name
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # bind something
   ?film wdt:P279 wd:Q10301427 .
   # get the label
   ?film sc:name ?name.
}
LIMIT 20
"""

# We have television pprogram IRI Q15416. Maybe television series is a subclass of it. Let's find out
queryString = """
SELECT DISTINCT ?tvs ?name
WHERE {
   # bind something
   ?tvs wdt:P279 wd:Q15416 .
   # get the label
   ?tvs sc:name ?name.
   FILTER regex(?name, ".*series.*", "i")
}
LIMIT 20
"""

# We found it! let's optimize the filter
queryString = """
SELECT DISTINCT ?tvs ?name
WHERE {
   # bind something
   ?tvs wdt:P279 wd:Q15416 .
   # get the label
   ?tvs sc:name ?name.
   FILTER regex(?name, "^television series$", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvs', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


Final query for this task

In [7]:
# write your final query
queryString = """
SELECT DISTINCT ?tvs ?name
WHERE {
   # bind something
   ?tvs wdt:P279 wd:Q15416 .
   # get the label
   ?tvs sc:name ?name.
   FILTER regex(?name, "^television series$", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tvs', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [8]:
# write your queries
# Let's fetch all films and tv series that are related to the canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?s ?name ?p ?pName
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
LIMIT 20
"""

# Now let's fetch the publication year
queryString = """
SELECT DISTINCT ?s ?name MIN(?date) as ?date
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P577 ?date} .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
GROUP BY ?s ?name
LIMIT 20
"""

# Some films are missing the publication year. Let's check if they the "incepion" property
queryString = """
SELECT DISTINCT ?s ?name MIN(?date) as ?date
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P577 ?date} .
   OPTIONAL {?s wdt:P571 ?date} .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
GROUP BY ?s ?name
LIMIT 20
"""

# Some dates are still missing. For the title Sherlock Holmes Q4523158 for example. let's see if we can find a property to use
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q4523158 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

# The property "start time" IRI P580 could be the solution. Let's try it
queryString = """
SELECT DISTINCT ?s ?name MIN(?date) as ?date
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P577 ?date} .
   OPTIONAL {?s wdt:P571 ?date} .
   OPTIONAL {?s wdt:P580 ?date} .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
GROUP BY ?s ?name
LIMIT 20
"""

# We have all the dates. Let's pick out the year only
queryString = """
SELECT DISTINCT ?s ?name MIN(year(?date) as ?date) as ?year
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P577 ?date} .
   OPTIONAL {?s wdt:P571 ?date} .
   OPTIONAL {?s wdt:P580 ?date} .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
GROUP BY ?s ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013')]
[('s', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('year', '2009')]
[('s', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2013')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('year', '2010')]
[('s', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('year', '1970')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984')]
[('s', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('year', '2011')]
7


Final query for this task

In [9]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name MIN(year(?date) as ?date) as ?year
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P577 ?date} .
   OPTIONAL {?s wdt:P571 ?date} .
   OPTIONAL {?s wdt:P580 ?date} .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
GROUP BY ?s ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013')]
[('s', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('year', '2009')]
[('s', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2013')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('year', '2010')]
[('s', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('year', '1970')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984')]
[('s', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('year', '2011')]
7


## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [10]:
# write your queries
# Let's use the 2009 "Sherlock Holmes" movie tofind the actors and actresses of it
queryString = """
SELECT DISTINCT ?ps ?psName
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   ?s ?ps ?obj .
   # get the label
   ?s sc:name ?name.
   ?ps sc:name ?psName.
}
LIMIT 20
"""

# The "cast member" property IRI P161 will give us what we need
queryString = """
SELECT DISTINCT ?act ?name count(?s) as ?works
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   ?s wdt:P161 ?act .
   # get the label
   ?act sc:name ?name.
}
GROUP BY ?act ?name
HAVING (count(?s) > 1)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('act', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('works', '2')]
7


Final query for this task

In [11]:
# write your final query
queryString = """
SELECT DISTINCT ?act ?name count(?s) as ?works
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   ?s wdt:P161 ?act .
   # get the label
   ?act sc:name ?name.
}
GROUP BY ?act ?name
HAVING (count(?s) > 1)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('act', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('works', '2')]
[('act', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('works', '2')]
7


## Task 6
Identify the BGP for Primetime Emmy Award

In [12]:
# write your queries
# Let's find all subjects that have Primetime Emmy Award in the name
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?s sc:name ?name.
   FILTER regex(?name, "Primetime Emmy Award", "i")
}
LIMIT 20
"""

# We found "Primetime Emmy Award for Outstanding Unstructured Reality Program" IRI Q18153303. Let's explore it
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q18153303 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

# It's in instance of Primetime Emmy Award IRI Q1044427
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q1044427 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

# Which in turn is an instanxe of Emmy Award IRI Q123737
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q123737 .
   # get the label
   ?award sc:name ?name.
   FILTER regex(?name, "^Primetime Emmy Award$", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
1


Final query for this task

In [13]:
# write your final query
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q123737 .
   # get the label
   ?award sc:name ?name.
   FILTER regex(?name, "^Primetime Emmy Award$", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
1


## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

In [14]:
# write your queries
# We just retunr all instances of Primetime Emmy Award 
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q1044427 .
   # get the label
   ?award sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('award', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('award', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('award', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('award', 'http://www.wikidata.org/en

Final query for this task

In [15]:
# write your final query
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q1044427 .
   # get the label
   ?award sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('award', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('award', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('award', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('award', 'http://www.wikidata.org/en

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [33]:
# write your queries
# Exploring the propeties connecting a film or series to the award
queryString = """
SELECT DISTINCT ?pa ?paname
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   ?s ?pa ?award .
   ?award wdt:P31 wd:Q1044427 .
   # get the label
   ?s sc:name ?name.
   ?pa sc:name ?paname.
}
LIMIT 20
"""

# The property "award received" IRI P166 is what we need
queryString = """
SELECT DISTINCT ?s ?name ?award
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P166 ?award . ?award wdt:P31 wd:Q1044427 } .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

# Let's group and count them
queryString = """
SELECT DISTINCT ?s ?name count(?award) as ?awards
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P166 ?award . ?award wdt:P31 wd:Q1044427 } .
   # get the label
   ?s sc:name ?name.
}
GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('awards', '0')]
[('s', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('awards', '0')]
[('s', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('awards', '0')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awards', '8')]
[('s', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('awards', '0')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('awards', '0')]
[('s', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('awards', '0')]
7


Final query for this task

In [34]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name count(?award) as ?award
WHERE {
   # bind something
   {?s wdt:P31 wd:Q5398426} UNION {?s wdt:P31 wd:Q11424} .
   ?s ?p wd:Q2316684 .
   OPTIONAL {?s wdt:P166 ?award . ?award wdt:P31 wd:Q1044427 } .
   # get the label
   ?s sc:name ?name.
}
GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('s', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('s', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('award', '0')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', '8')]
[('s', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('award', '0')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('s', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
7
